### Full algorithm for getting ready German data
HCHS_data(cleaning) -> replace abbrviations -> translation with Hugging Face -> rid of unnecessory information

#### HCHS data

In [35]:
import numpy as np
import pandas as pd
import spacy
from tqdm import tqdm
from transformers import pipeline

In [17]:
#Loading Soarian HCHS data
HCHS_data_second=pd.read_excel('/workspaces/de.uke.iam.automapping/experiments/VM_ _HCHS_10000_20210708.xlsx')
HCHS_data_second['Langname']=HCHS_data_second['Langname'].replace([' ', ''], np.nan)
HCHS_Langname_second=HCHS_data_second['Langname'].str.strip()
HCHS_Langname_second=HCHS_Langname_second.dropna(axis=0)

In [18]:
#Loading Fragebogen data
HCHS_data=pd.read_excel('/workspaces/de.uke.iam.automapping/experiments/VM_Soarian_HCHS_20210422.xlsx')
HCHS_data['Langname']=HCHS_data['Langname'].replace([' ', ''], np.nan)
HCHS_Langname=HCHS_data['Langname'].str.strip()
HCHS_Langname=HCHS_Langname.dropna(axis=0)

In [19]:
#Getting them together
HCHS_Langname= HCHS_Langname.append(HCHS_Langname_second, ignore_index = True)

/tmp/ipykernel_395/81917566.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  HCHS_Langname= HCHS_Langname.append(HCHS_Langname_second, ignore_index = True)


In [20]:
HCHS_Langname

0       Liegt bei Ihnen eine vom Arzt diagnostizierte ...
1       Liegt bei Ihnen ein vom Arzt diagnostizierter ...
2                          Wann war die letzte Injektion?
3                          Wann war die letzte Injektion?
4       Hatten Sie bereits einen vom Arzt diagnostizie...
                              ...                        
2669                                Sonstiges |  nämlich:
2670    Ich habe das Gefühl |  dass ich über Sexualitä...
2671           Ich bin motiviert |  sexuell aktiv zu sein
2672    Sexualität ist in meinem Leben eher durch Umst...
2673    In Bezug auf sexuelle Kontakte bin ich ängstli...
Name: Langname, Length: 2674, dtype: object

In [21]:
df=pd.DataFrame()
df['HCHS_German']=HCHS_Langname

In [22]:
df['HCHS_German']=df['HCHS_German'].str.replace(r'[{4}|]', ',', regex=True)#cases with |

#### Abbriviation replacement

In [23]:
list_abbr=pd.read_excel('/workspaces/de.uke.iam.automapping/experiments/german_abbreviation.xlsx')

In [24]:
df['HCHS_German_without_abbreviation']=df['HCHS_German']

In [25]:
#abbreviation replacment
for i in tqdm(range(list_abbr.shape[0])):
    abbreviation=list_abbr['Abbreviation'][i]
    description=list_abbr['Description'][i]
    df['HCHS_German_without_abbreviation']=df['HCHS_German_without_abbreviation'].str.replace(r'\b' + abbreviation + r'[^\w]' ,description+' ',  regex= True)

100%|██████████| 874/874 [00:17<00:00, 48.89it/s]


#### Translation German-English

In [26]:
model="Helsinki-NLP/opus-mt-de-en"
translator=pipeline('translation', model)

In [27]:
tr=translator(df['HCHS_German_without_abbreviation'].to_list())

In [28]:
translation_list=[]
for i in tr:
    translation_list.append(i.get('translation_text'))

In [29]:
df['HCHS_English']=translation_list

#### Rid of unnecessory information

In [ ]:
nlp=spacy.load('en_core_web_lg')
nlp.Defaults.stop_words.remove('no')
nlp.Defaults.stop_words.remove('not')
nlp.Defaults.stop_words.remove('none')
nlp.Defaults.stop_words.remove('noone')
nlp.Defaults.stop_words.remove('back')
nlp.Defaults.stop_words.add('doctors')

In [ ]:
from preprocessing import delete_extra

In [ ]:
upd_terms=delete_extra(df['HCHS_English'].str.lower().tolist(), nlp)

In [34]:
df.to_excel('20220625_HCHS_selected.xlsx', encoding='utf-16')